In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

# import warnings
# warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 200)

## Подготовка данных

In [50]:
df_osm = pd.read_csv('../../checkpoint-2/train_test/enriched_data.csv')
df_osm = df_osm.drop(columns = ['address','address_rus','geometry','utm'])
df_ira = pd.read_csv('../features_new/Dist_Area_features/df_dist_united.csv')[['id','dist','area']]

df = df_osm.merge(df_ira, on = ['id']).drop(columns = ['id'])
df['nearest_railway'][df['nearest_railway'].isna()] = df['nearest_railway'].max()
df['nearest_college'][df['nearest_college'].isna()] = df['nearest_college'].max()
df['nearest_office'][df['nearest_office'].isna()] = df['nearest_office'].max()
df['nearest_airport'][df['nearest_airport'].isna()] = df['nearest_airport'].max()
df['nearest_gov_building'][df['nearest_gov_building'].isna()] = df['nearest_gov_building'].max()
df['nearest_MFC'][df['nearest_MFC'].isna()] = df['nearest_MFC'].max()
df['nearest_subway'][df['nearest_subway'].isna()] = df['nearest_subway'].max()
df['nearest_neighbour_atm'][df['nearest_neighbour_atm'].isna()] = df['nearest_neighbour_atm'].max()


# Данные с ohe с нормирокой 
# Данные без ohe с нормировкой
# Данные с ohe без нормировки
# Данные без ohe без нормировки 

/var/folders/8m/fpt3kb2n0ys89v3nl34fszrd2jtv8f/T/ipykernel_40780/4044311551.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nearest_railway'][df['nearest_railway'].isna()] = df['nearest_railway'].max()
/var/folders/8m/fpt3kb2n0ys89v3nl34fszrd2jtv8f/T/ipykernel_40780/4044311551.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nearest_college'][df['nearest_college'].isna()] = df['nearest_college'].max()
/var/folders/8m/fpt3kb2n0ys89v3nl34fszrd2jtv8f/T/ipykernel_40780/4044311551.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documenta

In [51]:
train = df[~df.target.isna()].reset_index(drop = True)
test = df[df.target.isna()].drop(columns = ['target']).reset_index(drop = True)

In [52]:
train.head(2)

,atm_group,lat,lon,target,fed_dist,region,nearest_bank,nearest_railway,nearest_food,nearest_hospital,nearest_atm,nearest_college,nearest_office,nearest_school,nearest_airport,nearest_gov_building,nearest_kgarden,nearest_MFC,nearest_subway,nearest_neighbour_atm,bank_in_500m,railway_in_500m,food_in_500m,hospital_in_500m,atm_in_500m,college_in_500m,office_in_500m,school_in_500m,airport_in_500m,gov_building_in_500m,kgarden_in_500m,MFC_in_500m,subway_in_500m,dist,area
0,32.0,46.940995,142.738319,0.01150,Дальневосточный федеральный округ,Сахалинская область,610.347831,1969.488428,688.341776,846.734329,551.515436,251122.962844,1086.634741,533.768021,27053.216831,704186.370847,1254.645460,53940.935525,1.711936e+06,171.577438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30,160311.25
1,32.0,46.937353,142.753348,0.02971,Дальневосточный федеральный округ,Сахалинская область,55.010198,2146.384235,197.712149,760.622620,349.867950,251733.829151,732.357512,1359.050288,27455.714731,704805.042737,79.951992,55148.993149,1.711936e+06,243.129061,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,17,144069.25


## ОНЕ и Нормировка и варианты

In [53]:
train1 = train.copy()
test1 = test.copy()

enc = OneHotEncoder(handle_unknown='ignore',drop = 'first')
enc.fit(train1[['atm_group','fed_dist','region']])

train1[enc.get_feature_names_out(['atm_group','fed_dist','region'])] = pd.DataFrame(enc.transform(train1[['atm_group','fed_dist','region']]).toarray())
test1[enc.get_feature_names_out(['atm_group','fed_dist','region'])] = pd.DataFrame(enc.transform(test1[['atm_group','fed_dist','region']]).toarray())
train1 = train1.drop(columns=['atm_group','fed_dist','region'])
test1 = test1.drop(columns=['atm_group','fed_dist','region'])

X_train, X_test, y_train, y_test = train_test_split(train1.drop(columns = ['target']),train1.target , test_size=0.2, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)

X_train = pd.DataFrame(scaler.transform(X_train),columns = X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test),columns = X_test.columns)
test1 = pd.DataFrame(scaler.transform(test1),columns = test1.columns)

X_train.to_csv('../data/ohe-norm/X_train.csv',index = False)
y_train.to_csv('../data/ohe-norm/y_train.csv',index = False)
X_test.to_csv('../data/ohe-norm/X_test.csv',index = False)
y_test.to_csv('../data/ohe-norm/y_test.csv',index = False)
test1.to_csv('../data/ohe-norm/test.csv',index = False)


/opt/homebrew/Caskroom/miniforge/base/envs/py39/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [58]:
train2 = train.copy()
test2 = test.copy()

enc = OneHotEncoder(handle_unknown='ignore',drop = 'first')
enc.fit(train2[['atm_group','fed_dist','region']])

train2[enc.get_feature_names_out(['atm_group','fed_dist','region'])] = pd.DataFrame(enc.transform(train2[['atm_group','fed_dist','region']]).toarray())
test2[enc.get_feature_names_out(['atm_group','fed_dist','region'])] = pd.DataFrame(enc.transform(test2[['atm_group','fed_dist','region']]).toarray())
train2 = train2.drop(columns=['atm_group','fed_dist','region'])
test2 = test2.drop(columns=['atm_group','fed_dist','region'])

X_train, X_test, y_train, y_test = train_test_split(train2.drop(columns = ['target']),train2.target , test_size=0.2, random_state=42)

X_train.to_csv('../data/ohe-no-norm/X_train.csv',index = False)
y_train.to_csv('../data/ohe-no-norm/y_train.csv',index = False)
X_test.to_csv('../data/ohe-no-norm/X_test.csv',index = False)
y_test.to_csv('../data/ohe-no-norm/y_test.csv',index = False)
test2.to_csv('../data/ohe-no-norm/test.csv',index = False)

/opt/homebrew/Caskroom/miniforge/base/envs/py39/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [55]:
train3 = train.copy()
test3 = test.copy()


X_train, X_test, y_train, y_test = train_test_split(train3.drop(columns = ['target']),train3.target , test_size=0.2, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train[X_train.columns.difference(['atm_group', 'region','fed_dist'])])

X_train[X_train.columns.difference(['atm_group', 'region','fed_dist'])] = scaler.transform(X_train[X_train.columns.difference(['atm_group', 'region','fed_dist'])])
X_test[X_test.columns.difference(['atm_group', 'region','fed_dist'])] = scaler.transform(X_test[X_test.columns.difference(['atm_group', 'region','fed_dist'])])
test3[test3.columns.difference(['atm_group', 'region','fed_dist'])] = scaler.transform(test3[test3.columns.difference(['atm_group', 'region','fed_dist'])])

X_train.to_csv('../data/no-ohe-norm/X_train.csv',index = False)
y_train.to_csv('../data/no-ohe-norm/y_train.csv',index = False)
X_test.to_csv('../data/no-ohe-norm/X_test.csv',index = False)
y_test.to_csv('../data/no-ohe-norm/y_test.csv',index = False)
test3.to_csv('../data/no-ohe-norm/test.csv',index = False)

In [56]:
train4 = train.copy()
test4 = test.copy()


X_train, X_test, y_train, y_test = train_test_split(train4.drop(columns = ['target']),train4.target , test_size=0.2, random_state=42)

X_train.to_csv('../data/no-ohe-no-norm/X_train.csv',index = False)
y_train.to_csv('../data/no-ohe-no-norm/y_train.csv',index = False)
X_test.to_csv('../data/no-ohe-no-norm/X_test.csv',index = False)
y_test.to_csv('../data/no-ohe-no-norm/y_test.csv',index = False)
test4.to_csv('../data/no-ohe-no-norm/test.csv',index = False)

## Baseline средним/медианным значением

In [57]:
print('rmse median baseline:',mean_squared_error(y_test, [y_train.median()]*y_test.size, squared=False))
print('rmse mean baseline:',mean_squared_error(y_test, [y_train.mean()]*y_test.size, squared=False))
print('mape median baseline:',mean_absolute_percentage_error(y_test, [y_train.median()]*y_test.size))
print('mape mean baseline:',mean_absolute_percentage_error(y_test, [y_train.mean()]*y_test.size))

rmse median baseline: 0.08843944093289292
rmse mean baseline: 0.08663733448406714
mape median baseline: 2.616892798046592
mape mean baseline: 1.027674440168153


## Наш baseline rmse: 0.087 (среднее значение по таргету)
## mape 102%